В этом файле показано взаисодействие клиента с сервисом модели.

Модель запущена в Docker-контейнере на вируальной машине. Обращение к модели идёт с машины хоста.

Адрес сервиса: `https://localhost`, порт - `8180`. Этот порт необходимо открыть в настройках виртуальной машины.

In [1]:
import requests
import pandas as pd
import dill

Функция для формирования запроса:

In [2]:
def send_json(data):
    body = {'text':data.tolist()}
    myurl = 'http://localhost:8180/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()

In [3]:
X_test = pd.read_csv("X_test.csv").squeeze()

In [4]:
X_test

0       dispute credit card company chase visa purchas...
1       issue chase credit billing act credit card act...
2       please help dispute credit reporting agency ba...
3       call cellphone today company telephone number ...
4       time year milage card chasein ii mileage card ...
                              ...                        
5263                                           phone call
5264    chase card inquiry pull credit report authoriz...
5265    hotel chain credit card order take advantage p...
5266    hello credit card credit card company service ...
5267    account identity theft chase yhe account colle...
Name: _source.complaint_what_happened, Length: 5268, dtype: object

Отправим запрос:

In [5]:
%%time 
response = send_json(X_test.iloc[:5])
print(response)

{'predict': [3, 3, 1, 1, 1], 'predict_proba': [[0.0014702405771024178, 0.007156692639851781, 0.003526176765452387, 0.9867247427070928, 0.0011221473105006061], [0.0007254716962689858, 0.19133626704831921, 0.0004357515649638371, 0.8066626158134167, 0.0008398938770312514], [0.0009057899144539515, 0.9537378586912623, 0.022425207240474147, 0.004523623888101759, 0.018407520265707786], [0.15747096647425607, 0.5650291805238369, 0.1024274438235782, 0.15157943072493332, 0.02349297845339551], [0.027324453600976163, 0.6551139983946362, 0.0020670693557730273, 0.3070333584862603, 0.008461120162354245]], 'success': True}
CPU times: total: 0 ns
Wall time: 2.13 s


Можно отправлять как несколько строк из датасета, так и одну:

In [6]:
%%time 
response = send_json(X_test.iloc[:1])
print(response)

{'predict': [3], 'predict_proba': [[0.0014702405771024178, 0.007156692639851781, 0.003526176765452387, 0.9867247427070928, 0.0011221473105006061]], 'success': True}
CPU times: total: 15.6 ms
Wall time: 2.02 s


Проверим сколько времени займёт получени ответа, если отправить весь датасет:

In [7]:
%%time 
response = send_json(X_test)

CPU times: total: 46.9 ms
Wall time: 3.64 s


Полученный ответ содержит предсказанный класс, вероятности принадлежности к классу, и статус предсказания:

In [8]:
pd.DataFrame(response).head()

,predict,predict_proba,success
0,3,"[0.0014702405771024178, 0.007156692639851781, ...",True
1,3,"[0.0007254716962689858, 0.19133626704831921, 0...",True
2,1,"[0.0009057899144539515, 0.9537378586912623, 0....",True
3,1,"[0.15747096647425607, 0.5650291805238369, 0.10...",True
4,1,"[0.027324453600976163, 0.6551139983946362, 0.0...",True
